In [7]:
import pandas as pd
import json
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
import sys
import os
sys.path.insert(0, '../')
from scripts_second_phase.tracking import MatchTracking 
from scripts_second_phase.pass_events import PassEvents

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Relou de toujours devoir aller chercher le nom du match correspondant dans statsbomb : création d'un mapping
tracking_to_event  = {
    #'g2312201' : 'ManCity_AstonVilla',
    'g2312135' : 'ManCity_Arsenal',
    'g2312152' : 'ManCity_LeicesterCity',
    'g2312166' : 'ManCity_Liverpool',
    'g2312183' : 'ManCity_Brighton',
    'g2312213' : 'ManCity_Tottenham'
}
event_to_tracking = {v: k for k, v in tracking_to_event.items()}
event_to_tracking

{'ManCity_Arsenal': 'g2312135',
 'ManCity_LeicesterCity': 'g2312152',
 'ManCity_Liverpool': 'g2312166',
 'ManCity_Brighton': 'g2312183',
 'ManCity_Tottenham': 'g2312213'}

In [9]:
game = 'ManCity_Liverpool'

In [10]:
if False :  
    # Vérification qu'on attaque bien toujours à droite dans les données de tracking
    match_tracking = MatchTracking(tracking_file=event_to_tracking[game], event_file=game)

    df_home = match_tracking.HomeTracking.df_tracking.copy()
    df_home1 = df_home[df_home['period']==1].sample(1000)
    plt.scatter(df_home1.x,df_home1.y)
    plt.show()

    df_home2 = df_home[df_home['period']==2].sample(1000)
    plt.scatter(df_home2.x,df_home2.y)
    plt.show()

    df_away = match_tracking.AwayTracking.df_tracking.copy()
    df_away1 = df_away[df_away['period']==1].sample(1000)
    plt.scatter(df_away1.x,df_away1.y)
    plt.show()

    df_away2 = df_away[df_away['period']==2].sample(1000)
    plt.scatter(df_away2.x,df_away2.y)
    plt.show()

In [11]:
if False : 
    # Vérification qu'on attaque bien toujours à droite dans les données de passes
    pass_events = PassEvents(event_file=game)

    df_home = pass_events.df_pass_home.copy()
    df_home1 = df_home[df_home['period']==1]
    plt.scatter(df_home1.x,df_home1.y)
    plt.show()

    df_home2 = df_home[df_home['period']==2]
    plt.scatter(df_home2.x,df_home2.y)
    plt.show()

    df_away = pass_events.df_pass_away.copy()
    df_away1 = df_away[df_away['period']==1]
    plt.scatter(df_away1.x,df_away1.y)
    plt.show()

    df_away2 = df_away[df_away['period']==2]
    plt.scatter(df_away2.x,df_away2.y)
    plt.show()

In [12]:
train_set = list()
for game in tqdm(event_to_tracking) :

    match_tracking = MatchTracking(tracking_file=event_to_tracking[game], event_file=game)
    pass_events = PassEvents(event_file=game)

    # Synchronisation de la position du passeur et receveur 
    pass_events.update_position(match_tracking=match_tracking)
    # Ajout de la position des autres joueurs
    pass_events.update_dataset_with_position(match_tracking=match_tracking)
    # Ajout dans l'objet de la dimension du terrain
    pass_events.get_pitch_dimensions(match_tracking=match_tracking)
    # Suprresion des passes inexploitables
    pass_events.clean_dataset()
    # Calcul des features
    pass_events.set_df_for_model()

    if game == 'ManCity_AstonVilla' :
        pass_events.df_model.to_csv('astonvilla.csv')
    else :
        train_set.append(pass_events.df_model)

train_set = pd.concat(train_set)
train_set.to_csv('pass_dataset.csv')

  0%|          | 0/5 [00:00<?, ?it/s]../scripts_second_phase/tracking.py:134: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  home_goalkeeper_diff_position = self.HomeTracking.df_tracking[
../scripts_second_phase/tracking.py:136: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  away_goalkeeper_diff_position = self.AwayTracking.df_tracking[
../scripts_second_phase/pass_events.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u